In [1]:
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy import units as u
from scipy import stats
from spectres import spectres
from inlifesim.bootstrapping import InterpretBootstrapping
from lifesim.util.radiation import black_body

import inlifesim as ils

# 1. Setup parameters and load spectrum

In [2]:
path_spectrum = 'data/Earth_PRTunits_10pc.txt'
path_bs = 'data/lookup_table_1e10_Nov24.pkl'

In [3]:
earth_twin = {'distance_s': 10,
              'temp_s': 5778.,
              'radius_s': 1.,
              'lat_s': 0.78,
              'l_sun': 1.,
              'z': 1.,
              'temp_p': 254.,
              'radius_p': 1.,
              'sep_p': 1.,}

dbw = {'ap_diameter': 3.,
       'spec_res': 50.,
       't_int': 16 * 24 * 60 * 60,  #  10 * 24 * 60 * 60
       'throughput': 0.035,
       't_exp': 60 * 10,
       'n_rot': 15,
       'wl_bin': 10e-6,  # 10e-6
       'bl': 14.5,
       'ratio': 6,
       'wl_bin_width': 0.3e-6,  # 0.3e-6
       'rms_mode': 'wavelength',
       'hyperrot_noise': 'zero',
       'd_phi_rms': 0.0013,  # 0.005, 0.001
       'd_a_rms': 0.0013,
       'd_x_rms': 0.,
       'd_y_rms': 0.,
       'd_pol_rms': 0.0013,
       'd_a_co': 10e3,
       'd_phi_co': 10e3,
       'd_pol_co': 10e3,
       'd_x_co': 0.64e-3,
       'd_y_co': 0.64e-3}

In [4]:
data = pd.read_csv(
    path_spectrum,
    header=None, sep='\s+')

lam_PRT = data[0].values * u.micron
f_PRT = data[1].values * u.erg / u.cm ** 2 / u.s / u.Hz
f_lifesim = f_PRT.to(u.photon / u.m ** 2 / u.s / u.micron,
                     equivalencies=u.spectral_density(lam_PRT))

f_lifesim = f_lifesim.to(u.photon / u.s / u.meter ** 3)
lam_lifesim = lam_PRT.to(u.meter)

# scale planet flux to distance
f_lifesim *= (10 / earth_twin['distance_s']) ** 2

flux_planet_spectrum = [lam_lifesim, f_lifesim]
f_lifesim = spectres(new_wavs=np.array((dbw['wl_bin'] - dbw['wl_bin_width'] / 2, dbw['wl_bin'] + dbw['wl_bin_width'] / 2)),
                     spec_wavs=lam_lifesim.value,
                     spec_fluxes=f_lifesim.value,
                     edge_mode=True)
flux_planet_spectrum = f_lifesim * dbw['wl_bin_width']

# 2. Run the instrument model

In [5]:
col_pos = np.array((
                (-dbw['bl'] / 2,
                 -dbw['bl'] * dbw['ratio'] / 2),
                (-dbw['bl'] / 2,
                 dbw['bl'] * dbw['ratio'] / 2),
                (dbw['bl'] / 2,
                 -dbw['bl'] * dbw['ratio'] / 2),
                (dbw['bl'] / 2,
                 dbw['bl'] * dbw['ratio'] / 2)
            ))

instrument = ils.Instrument(wl_bins=np.array((dbw['wl_bin'], )),
                            wl_bin_widths=np.array((dbw['wl_bin_width'], )),
                            image_size=512,
                            diameter_ap=dbw['ap_diameter'],
                            flux_division=np.array((0.25, 0.25, 0.25, 0.25)),
                            throughput=dbw['throughput'],
                            dist_star=earth_twin['distance_s'],
                            radius_star=earth_twin['radius_s'],
                            temp_star=earth_twin['temp_s'],
                            lat_star=earth_twin['lat_s'],
                            l_sun=earth_twin['l_sun'],
                            z=earth_twin['z'],
                            temp_planet=earth_twin['temp_p'],
                            radius_planet=earth_twin['radius_p'],
                            separation_planet=earth_twin['sep_p'],
                            col_pos=col_pos,
                            phase_response=np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            phase_response_chop=-np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            n_rot=dbw['n_rot'],
                            t_total=dbw['t_int'],
                            t_exp=dbw['t_exp'],
                            n_cpu=1,
                            rms_mode=dbw['rms_mode'],
                            hyperrot_noise=dbw['hyperrot_noise'],
                            n_sampling_max=int(1e7),
                            d_a_rms=dbw['d_a_rms'],
                            d_phi_rms=dbw['d_phi_rms'],
                            d_pol_rms=dbw['d_pol_rms'],
                            d_x_rms=dbw['d_x_rms'],
                            d_y_rms=dbw['d_y_rms'],
                            d_a_co=dbw['d_a_co'],
                            d_phi_co=dbw['d_phi_co'],
                            d_pol_co=dbw['d_pol_co'],
                            d_x_co=dbw['d_x_co'],
                            d_y_co=dbw['d_y_co'],
                            n_draws=None,
                            n_draws_per_run=None,
                            time_series_return_values='all',
                            flux_planet=flux_planet_spectrum,
                            simultaneous_chopping=True,
                            verbose=True,
                            draw_samples=False,
                            get_single_bracewell=False,
                            instrumental_source=None)
instrument.run()

ils_res = deepcopy(instrument.photon_rates_chop)

Adjusted exposure time from 600 s to 594.58 s
Will simulate 15 rotations in 16.0 days
Total number of samples: 2325
Number of rotation angles: 2325
Creating astrophysical sources ... [Done]
Calculating gradient and Hessian coefficients ... [Done]
Generating planet signal ... [Done]
Shape of the planet template: (1, 2325)
Calculating fundamental noise ... [Done]
Calculating systematics noise (chopping) ... [Done]


To diffenretiate between noise sources, the instrument model needs to be run mutliple times

In [6]:
instrument_star = ils.Instrument(wl_bins=np.array((dbw['wl_bin'], )),
                            wl_bin_widths=np.array((dbw['wl_bin_width'], )),
                            image_size=512,
                            diameter_ap=dbw['ap_diameter'],
                            flux_division=np.array((0.25, 0.25, 0.25, 0.25)),
                            throughput=dbw['throughput'],
                            dist_star=earth_twin['distance_s'],
                            radius_star=earth_twin['radius_s'],
                            temp_star=earth_twin['temp_s'],
                            lat_star=earth_twin['lat_s'],
                            l_sun=earth_twin['l_sun'],
                            z=earth_twin['z'],
                            temp_planet=earth_twin['temp_p'],
                            radius_planet=earth_twin['radius_p'],
                            separation_planet=earth_twin['sep_p'],
                            col_pos=col_pos,
                            phase_response=np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            phase_response_chop=-np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            n_rot=dbw['n_rot'],
                            t_total=dbw['t_int'],
                            t_exp=dbw['t_exp'],
                            n_cpu=1,
                            rms_mode=dbw['rms_mode'],
                            hyperrot_noise=dbw['hyperrot_noise'],
                            n_sampling_max=int(1e7),
                            d_a_rms=dbw['d_a_rms'],
                            d_phi_rms=dbw['d_phi_rms'],
                            d_pol_rms=dbw['d_pol_rms'],
                            d_x_rms=dbw['d_x_rms'],
                            d_y_rms=dbw['d_y_rms'],
                            d_a_co=dbw['d_a_co'],
                            d_phi_co=dbw['d_phi_co'],
                            d_pol_co=dbw['d_pol_co'],
                            d_x_co=dbw['d_x_co'],
                            d_y_co=dbw['d_y_co'],
                            n_draws=None,
                            n_draws_per_run=None,
                            time_series_return_values='all',
                            flux_planet=flux_planet_spectrum,
                            simultaneous_chopping=True,
                            verbose=True,
                            draw_samples=False,
                            get_single_bracewell=False,
                            instrumental_source='star')
instrument_star.run()

ils_res_star = deepcopy(instrument_star.photon_rates_chop)

instrument_ez = ils.Instrument(wl_bins=np.array((dbw['wl_bin'], )),
                            wl_bin_widths=np.array((dbw['wl_bin_width'], )),
                            image_size=512,
                            diameter_ap=dbw['ap_diameter'],
                            flux_division=np.array((0.25, 0.25, 0.25, 0.25)),
                            throughput=dbw['throughput'],
                            dist_star=earth_twin['distance_s'],
                            radius_star=earth_twin['radius_s'],
                            temp_star=earth_twin['temp_s'],
                            lat_star=earth_twin['lat_s'],
                            l_sun=earth_twin['l_sun'],
                            z=earth_twin['z'],
                            temp_planet=earth_twin['temp_p'],
                            radius_planet=earth_twin['radius_p'],
                            separation_planet=earth_twin['sep_p'],
                            col_pos=col_pos,
                            phase_response=np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            phase_response_chop=-np.array((0, np.pi / 2, np.pi, 3 * np.pi / 2)),
                            n_rot=dbw['n_rot'],
                            t_total=dbw['t_int'],
                            t_exp=dbw['t_exp'],
                            n_cpu=1,
                            rms_mode=dbw['rms_mode'],
                            hyperrot_noise=dbw['hyperrot_noise'],
                            n_sampling_max=int(1e7),
                            d_a_rms=dbw['d_a_rms'],
                            d_phi_rms=dbw['d_phi_rms'],
                            d_pol_rms=dbw['d_pol_rms'],
                            d_x_rms=dbw['d_x_rms'],
                            d_y_rms=dbw['d_y_rms'],
                            d_a_co=dbw['d_a_co'],
                            d_phi_co=dbw['d_phi_co'],
                            d_pol_co=dbw['d_pol_co'],
                            d_x_co=dbw['d_x_co'],
                            d_y_co=dbw['d_y_co'],
                            n_draws=None,
                            n_draws_per_run=None,
                            time_series_return_values='all',
                            flux_planet=flux_planet_spectrum,
                            simultaneous_chopping=True,
                            verbose=True,
                            draw_samples=False,
                            get_single_bracewell=False,
                            instrumental_source='ez')
instrument_ez.run()

ils_res_ez = deepcopy(instrument_ez.photon_rates_chop)

Adjusted exposure time from 600 s to 594.58 s
Will simulate 15 rotations in 16.0 days
Total number of samples: 2325
Number of rotation angles: 2325
Creating astrophysical sources ... [Done]
Calculating gradient and Hessian coefficients ... [Done]
Generating planet signal ... [Done]
Shape of the planet template: (1, 2325)
Calculating fundamental noise ... [Done]
Calculating systematics noise (chopping) ... [Done]
Adjusted exposure time from 600 s to 594.58 s
Will simulate 15 rotations in 16.0 days
Total number of samples: 2325
Number of rotation angles: 2325
Creating astrophysical sources ... [Done]
Calculating gradient and Hessian coefficients ... [Done]
Generating planet signal ... [Done]
Shape of the planet template: (1, 2325)
Calculating fundamental noise ... [Done]
Calculating systematics noise (chopping) ... [Done]


# 3. Evaluate results from instrument model

In [7]:
total_random = np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2 + ils_res['pn_snfl'].values[0]) / dbw['t_int']

total_syst = np.sqrt(ils_res['sn_fo'].values[0] + ils_res['sn_so'].values[0]) / dbw['t_int']
total_noise = np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2 + ils_res['pn_snfl'].values[0] + ils_res['sn_fo'].values[0] + ils_res['sn_so'].values[0]) / dbw['t_int']
signal = ils_res['signal'].values[0] / dbw['t_int']

test_stat = ils_res['signal'][0] / np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2 + ils_res['pn_snfl'].values[0] + ils_res['sn_fo'].values[0] + ils_res['sn_so'].values[0])

sigma_imb_gauss = np.sqrt(ils_res['sn_so'].values[0])/np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2 + ils_res['pn_snfl'].values[0] + ils_res['sn_fo'].values[0])

/var/folders/rs/chvp5qfj3j76n4gdp708g4vc0000gn/T/ipykernel_4546/1424824848.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_stat = ils_res['signal'][0] / np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2 + ils_res['pn_snfl'].values[0] + ils_res['sn_fo'].values[0] + ils_res['sn_so'].values[0])


In [12]:
ib = InterpretBootstrapping(path_bs)
sigma_actual = ib.get_sigma_actual(sigma_analytical=np.array((test_stat, )),
                                   sigma_ratio=np.array((sigma_imb_gauss, )))
print('T_N: {:.2f}'.format(sigma_actual[0]))

T_N: 7.90


In [9]:
source_total = {'star': float(np.sqrt(ils_res_star['pn_sgl'].values[0]**2 + ils_res_star['pn_snfl'] + ils_res_star['sn_fo'] + ils_res_star['sn_so']) / dbw['t_int']),
                'exo': 
            float(np.sqrt(ils_res_ez['pn_ez'].values[0]**2 + ils_res_ez['pn_snfl'] + ils_res_ez['sn_fo'] + ils_res_ez['sn_so']) / dbw['t_int']),
                'local': 
            ils_res['pn_lz'].values[0] / dbw['t_int'],
                }

kind_total = {
            'fundamental': np.sqrt(ils_res['pn_sgl'].values[0]**2 + ils_res['pn_ez'].values[0]**2 + ils_res['pn_lz'].values[0]**2) / dbw['t_int'],
            'instrumental': np.sqrt((ils_res['pn_snfl'].values[0])) / dbw['t_int'],
            'first_order': np.sqrt(ils_res['sn_fo'].values[0]) / dbw['t_int'],
            'second_order': np.sqrt(ils_res['sn_so'].values[0]) / dbw['t_int']
}

/var/folders/rs/chvp5qfj3j76n4gdp708g4vc0000gn/T/ipykernel_4546/1084792482.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  source_total = {'star': float(np.sqrt(ils_res_star['pn_sgl'].values[0]**2 + ils_res_star['pn_snfl'] + ils_res_star['sn_fo'] + ils_res_star['sn_so']) / dbw['t_int']),
/var/folders/rs/chvp5qfj3j76n4gdp708g4vc0000gn/T/ipykernel_4546/1084792482.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(np.sqrt(ils_res_ez['pn_ez'].values[0]**2 + ils_res_ez['pn_snfl'] + ils_res_ez['sn_fo'] + ils_res_ez['sn_so']) / dbw['t_int']),


In [10]:
print('Total random noise: {:.1e} ph s-1'.format(total_random))
print('Fundamental random noise: {:.1e} ph s-1'.format(kind_total['fundamental']))
print('Instrumental random noise: {:.1e} ph s-1'.format(kind_total['instrumental']))
print('')
print('Total systematic noise: {:.1e} ph s-1'.format(total_syst))
print('First order systematic noise: {:.1e} ph s-1'.format(kind_total['first_order']))
print('Second order systematic noise: {:.1e} ph s-1'.format(kind_total['second_order']))
print('')

print('Total noise: {:.1e} ph s-1'.format(total_noise))
print('Signal: {:.1e} ph s-1'.format(signal))
print('')

print('T_alpha: {:.2f}'.format(test_stat))
print('Variance ratio: {:.2f}'.format(sigma_imb_gauss))

Total random noise: 6.1e-03 ph s-1
Fundamental random noise: 6.1e-03 ph s-1
Instrumental random noise: 2.0e-05 ph s-1

Total systematic noise: 5.8e-03 ph s-1
First order systematic noise: 3.3e-04 ph s-1
Second order systematic noise: 5.8e-03 ph s-1

Total noise: 8.4e-03 ph s-1
Signal: 7.1e-02 ph s-1

T_alpha: 8.42
Variance ratio: 0.96


In [11]:
result_table = pd.DataFrame(
    columns=['Fundamental',
             'Instrumental',
             'First order',
             'Second order'],
    index=['Star', 'Exo-zodi', 'Local-zodi'],
    data=[[
        ils_res['pn_sgl'].values[0] / dbw['t_int'],
        np.sqrt(ils_res_star['pn_snfl'].values[0]) / dbw['t_int'],
        np.sqrt(ils_res_star['sn_fo'].values[0]) / dbw['t_int'],
        np.sqrt(ils_res_star['sn_so'].values[0]) / dbw['t_int']
    ],
        [
            ils_res['pn_ez'].values[0] / dbw['t_int'],
            np.sqrt(ils_res_ez['pn_snfl'].values[0]) / dbw['t_int'],
            np.sqrt(ils_res_ez['sn_fo'].values[0]) / dbw['t_int'],
            np.sqrt(ils_res_ez['sn_so'].values[0]) / dbw['t_int']
        ],
        [
            ils_res['pn_lz'].values[0] / dbw['t_int'],
            '-',
            '-',
            '-'
        ]]
)

# Function to format numbers in scientific notation
def format_scientific(value):
    if isinstance(value, str):  # Keep dashes as is
        return value
    return "{:.1e}".format(value)

# Use applymap for formatting
result_table = result_table.applymap(format_scientific)

# Output the result table
print(result_table)

           Fundamental Instrumental First order Second order
Star           4.4e-03      2.0e-05     3.3e-04      5.8e-03
Exo-zodi       1.7e-03      1.2e-06     1.2e-07      1.3e-09
Local-zodi     3.8e-03            -           -            -


/var/folders/rs/chvp5qfj3j76n4gdp708g4vc0000gn/T/ipykernel_4546/45013794.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  result_table = result_table.applymap(format_scientific)
